In [1]:
import json
import numpy as np
import pandas as pd
import re
import torch

from transformers import AutoModelForSequenceClassification, AutoTokenizer
from pathlib import Path

In [2]:
torch.cuda.is_available()

True

In [3]:
df = pd.read_json("./results/multinli_transforms_bertscore_dev_matched.jsonl", lines=True)
df = df[df['gold_label'] != '-']

In [4]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

sentence_pairs = [(row['sentence1'], row['sentence2']) for index, row in list(df.iterrows())]
sentence_pairs_transformed = [
    [row['sentence1_transform1'] if row['sentence1_transform1_bertscore'] > row['sentence1_transform2_bertscore'] else row['sentence1_transform2'],
     row['sentence2_transform1'] if row['sentence2_transform1_bertscore'] > row['sentence2_transform2_bertscore'] else row['sentence2_transform2']] 
    for index, row in list(df.iterrows())
]

labels = []

In [5]:
# pose sequence as a NLI premise and label as a hypothesis
nli_model = AutoModelForSequenceClassification.from_pretrained('cross-encoder/nli-deberta-v3-large').to(device)
tokenizer = AutoTokenizer.from_pretrained('cross-encoder/nli-deberta-v3-large')

# batch to prevent memory issues
for i in range(0, len(sentence_pairs), 50):
    print(i)
    sentence_pair_batch = sentence_pairs_transformed[i:i+50]

    # run through model pre-trained on MNLI
    features = tokenizer(sentence_pair_batch, padding=True, truncation=True, return_tensors='pt').to(device)

    nli_model.eval()
    with torch.no_grad():
        scores = nli_model(**features).logits
        label_mapping = ['contradiction', 'entailment', 'neutral']
        labels.extend([label_mapping[score_max] for score_max in scores.argmax(dim=1)])
    
    del features
    torch.cuda.empty_cache()

/home/erwin/.virtualenvs/thesis/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:454: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450
2500
2550
2600
2650
2700
2750
2800
2850
2900
2950
3000
3050
3100
3150
3200
3250
3300
3350
3400
3450
3500
3550
3600
3650
3700
3750
3800
3850
3900
3950
4000
4050
4100
4150
4200
4250
4300
4350
4400
4450
4500
4550
4600
4650
4700
4750
4800
4850
4900
4950
5000
5050
5100
5150
5200
5250
5300
5350
5400
5450
5500
5550
5600
5650
5700
5750
5800
5850
5900
5950
6000
6050
6100
6150
6200
6250
6300
6350
6400
6450
6500
6550
6600
6650
6700
6750
6800
6850
6900
6950
7000
7050
7100
7150
7200
7250
7300
7350
7400
7450
7500
7550
7600
7650
7700
7750
7800
7850
7900
7950
8000
8050
8100
8150
8200
8250
8300
8350
8400
8450
8500
8550
8600
8650
8700
8750
8800
8850
8900
8950
9000
9050
9100
9150
9200
9250
9300
9350
9400
9450
9500
9550
9600
9650
9700
9750
9800


In [6]:
print(len(labels))

9815


In [7]:
gold_labels = df['gold_label'].to_list()

In [8]:
accuracy = sum([prediction == gold_label for prediction, gold_label in zip(labels, gold_labels)]) / len(labels)

In [9]:
print(accuracy)

0.7833927661742232


In [10]:
df['deberta_large_transform_label'] = labels
new_df = df[['pairID', 'deberta_large_transform_label']]

In [11]:
new_df.shape

(9815, 2)

In [12]:
save_path = Path("./results/deberta_large_transform_labels.jsonl")
new_df.to_json(save_path, orient="records", lines=True)